In [135]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import ta
from datapackage import Package
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import os
import random
import shutil
import time

In [40]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
response.status_code

200

In [51]:
soup = BeautifulSoup(response.content)
here_are_tickers = soup.find_all("a", attrs={"class":"external text"}) #this is the part that contains ticker code, but it is not unique, so further cleaning is needed
len(here_are_tickers) #checking how many items I have (list should be around 500, of course, although the SP500 always has slightly more than 500 members)
tickers = [t.text for t in ticker_list if t.text == t.text.upper()] #extracting only the members that are uppercase, which should be just the tickers
len(tickers) #503 -> The number looks right!

503

In [63]:
# this goes through the ticker list and creates the corresponding CSVs (dots in tickers are replaced by dashes so the API parses them properly)
for code in tickers:
	obj = yf.Ticker(code.replace(".", "-"))
	history = obj.history(period="max")
	history.to_csv(code + ".csv")

Got error from yahoo api for ticker BRK.B, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- BRK.B: No data found for this date range, symbol may be delisted
- BF.B: 1d data not available for startTime=-2208994789 and endTime=1669158538. Only 100 years worth of day granularity data are allowed to be fetched per request.


In [105]:
#Creates a Panda DataFrame where we append *all* data points, including a column which tells us which stock the data comes from
# Then saves to both csv and pickle formats

big_table = pd.DataFrame(columns=["Date", "Close", "Volume", "Code"])
for i in os.listdir("data"):
	little_table = pd.read_csv("data/" + i)
	little_table['Code'] = i.replace(".csv", "")
	big_table = pd.concat([big_table, little_table[["Date", "Close", "Volume", "Code"]]], axis=0)
big_table.to_csv("big_table.csv")
pd.to_pickle(big_table, "all_data.pkl")